# Hard
- Test Environment: Ubuntu 18.04 x86 py-visa, NI-VISA


## TODO:

Testing on more platforms:

- x86, ubuntu-18.04.
- raspberrypi4.
- FreeBSD.

In [2]:
import time
import mhs5200

In [33]:
import platform

In [40]:
import os

In [41]:
os.name

'posix'

In [39]:
platform.win32_ver()

('', '', '', '')

In [23]:
if os.name == "posix":
    signal_gen_port = "/dev/ttyUSB0"
    oscilloscope_port = scope_port = 'USB0::6833::1416::DS1EU152500705\x00::0::INSTR'
if os.name == "nt"
    signal_gen_port = "COM4"
    oscilloscope_port = scope_port = 'USB0::0x1AB1::0x0588::DS1EU152500705::INSTR'

In [24]:
import mhs5200.enums

In [25]:
with mhs5200.MHS5200(signal_gen_port) as signal_generator:
    for channel in signal_generator.channels:
        channel.amplitude=1
        channel.duty_cycle=50
        channel.frequency=1
        channel.offset=0
        channel.phase=0
        channel.wave=mhs5200.enums.SQUARE

In [26]:
import pyvisa
rm = pyvisa.ResourceManager('@py')
print(rm.list_resources())

Found a device whose serial number cannot be read. The partial VISA resource name is: USB0::6833::1416::???::0::INSTR


('ASRL/dev/ttyS4::INSTR', 'ASRL/dev/ttyUSB0::INSTR')


In [29]:
rm = pyvisa.ResourceManager()
rm.list_resources()

('ASRL/dev/ttyS4::INSTR',
 'ASRL/dev/ttyUSB0::INSTR',
 'USB0::6833::1416::DS1EU152500705\x00::0::INSTR')

In [32]:
rm.open_resource('USB0::6833::1416::DS1EU152500705\x00::0::INSTR')

<'USBInstrument'('USB0::6833::1416::DS1EU152500705\x00::0::INSTR')>

In [ ]:
import uuid

In [ ]:
class Scope():
    def __init__(self, *args, oscilloscope=None, oscilloscope_port=None):
        if oscilloscope_port is not None:
            self.resource = rm.open_resource(oscilloscope_port)
        else:
            self.resource = oscilloscope
            
        for fcn in ["write", "query", "close", "query_ascii_values"]:
            setattr(self, fcn, getattr(self.resource, fcn))
        
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, tb):
        t1=time.time()
        try:
            while True:
                if time.time()>t1+5:
                    break
                self.close()
                time.sleep(0.2)
        except InvalidSession:
            pass
        except:
            raise

In [ ]:
scope = Scope(oscilloscope_port=oscilloscope_port)

In [ ]:
scope = Scope(oscilloscope_port=oscilloscope_port)

In [ ]:
scope.close()

In [ ]:
import pandas as pd

In [ ]:
def test_frequency_amplitude3(frequency, amplitude, signal_gen, scope):
    for chan in signal_gen.channels:
        chan.frequency=frequency
        chan.amplitude=amplitude
        chan.phase=0
    
    period = 1/float(frequency)
    timescale="{:.20f}".format(float(period/5))
    # Configure scope
    
    scope.write(f":MEASURE:TOTAL ON")
    scope.write(f":TIMebase:SCALE {timescale}")
    for scope_channel in [1, 2]:
        scope.write(f":CHANNEL{scope_channel}:probe 1")
        scope.write(f":CHANNEL{scope_channel}:scale {amplitude/5}")
        scope.write(f":CHANNEL{scope_channel}:offset 0")
    # Configure signal generator
    for chan in signal_gen.channels:
        chan.frequency=frequency
        chan.amplitude=amplitude
        chan.offset = 0
        chan.phase=0

    df = dict()
    df["uuid"] = str(uuid.uuid4())
    df["frequency"] = frequency
    df["amplitude"] = amplitude
    for source in ["CHAN1", "CHAN2"]:
        scope.write(f":MEASURE:SOURCE {source}")
        time.sleep(1)
        for param in ['VPP',
                        'VMAX',
                        'VMIN',
                        'VAMPlitude',
                        'VTOP',
                        'VBASe',
                        'VAVerage',
                        'VRMS',
                        'OVERshoot',
                        'PREShoot',
                        'FREQuency',
                        'RISetime',
                        'FALLtime',
                        'PERiod',
                        'PWIDth',
                        'NWIDth',
                        'PDUTycycle',
                        'NDUTycycle',
                        'PDELay',
                        'NDELay',
                        'TOTal',
                        'SOURce',]:
            try:
                measured = scope.query_ascii_values(f":MEASURE:{param}?")[0]
            except:
                measured = scope.query(f":MEASURE:{param}?")[0]

            df[f"{source}_{param}"] = measured
    return pd.DataFrame(df, index=[0])

Loop through:

$$f=[1,10,100]$$
$$A=[1,10,20]$$

10x times to create statistics.

In [ ]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for _ in range(10):
            for frequency in [1, 10, 100]:
                for amplitude in [1, 10, 20]:
                    result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                    df = df.append(result_df)

In [ ]:
df.groupby(["frequency", "amplitude"]).agg(["mean", "median", "std", "var"])

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for _ in range(2):
            for frequency in np.logspace(np.log10(1000), np.log10(1000000), 2):
                for amplitude in [20]:
                    result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                    df = df.append(result_df)

In [ ]:
df.groupby(["frequency"])["CHAN1_FREQuency"].agg(["mean", "median", "std", "var"])

In [ ]:
df.groupby(["frequency", "amplitude"])["CHAN2_FREQuency"].agg(["mean", "median", "std", "var"])

In [ ]:
df = pd.DataFrame()
with mhs5200.MHS5200(signal_gen_port) as signal_gen:
    with Scope(oscilloscope_port=oscilloscope_port) as scope:
        for _ in range(2):
            for frequency in np.logspace(np.log10(1000000), np.log10(100000000), 10):
                for amplitude in [20]:
                    result_df = test_frequency_amplitude3(frequency, amplitude, signal_gen=signal_gen, scope=scope)
                    df = df.append(result_df)
df.groupby(["frequency"])["CHAN1_FREQuency"].agg(["mean", "median", "std", "var"])